**Часть 3. Классификация текстов**

**Сформулируем для простоты задачу бинарной классификации: будем классифицировать на два класса, то есть, различать резко отрицательные отзывы (с оценкой 1) и положительные отзывы (с оценкой 5).**

**1. Составьте обучающее и тестовое множество: выберите из всего набора данных N1 отзывов с оценкой 1 и N2 отзывов с оценкой 5 (значение N1 и N2 – на ваше усмотрение). Используйте sklearn.model_selection.train_test_split для разделения множества отобранных документов на обучающее и тестовое.**

In [ ]:
import json
import bz2
from tqdm import tqdm

In [ ]:
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
responses = []
with bz2.BZ2File('banki_responses.json.bz2', 'r') as thefile:
    for row in tqdm(thefile):
        resp = json.loads(row)
        if not resp['rating_not_checked'] and (len(resp['text'].split()) > 0):
            responses.append(resp)

201030it [01:08, 2925.69it/s]


In [ ]:
responses[:1]

[{'city': 'г. Москва',
  'rating_not_checked': False,
  'title': 'Жалоба',
  'num_comments': 0,
  'bank_license': 'лицензия № 2562',
  'author': 'uhnov1',
  'bank_name': 'Бинбанк',
  'datetime': '2015-06-08 12:50:54',
  'text': 'Добрый день! Я не являюсь клиентом банка и поручителем по кредитному договору, а также не являюсь каким-либо другим лицом, письменно  оформившим отношения с банком по поводу урегулирования чьей-либо  задолженности.  Начиная с марта 2015 года начали приходить бесконечные письма из ООО "Примо коллект"на мой адрес: город Москва, Уваровский переулок, дом 10, квартира 111, с угрозами о возбуждении уголовного дела в отношении гражданина Филиппова Эдуарда Владимировича, который уклоняется от уплаты взятых им кредитов: договор № 81014 от 20.10.2013 года и договор № 2464946 от 09.10.2014 года. Со всей ответственностью\xa0 хочу Вас заверить, что вышеуказанный гражданин, которого Вы разыскиваете, мне не знаком и никогда в моем адресе не был зарегистрирован. Каким образом 

In [ ]:
import re

In [ ]:
# Функция для предобработки текстов отзывов:
def cleaning(response):

  resp_ = re.sub(r'http\S+', '', response) # удаление ссылок
  resp_ = re.sub('-', ' ', resp_) # замена дефисов на пробелы
  resp_ = resp_.lower() # приведение к нижнему регистру
  resp_ = re.sub('\s+', ' ', resp_) # удаление лишних пробельных символов
  resp_ = re.sub('[^а-яёА-Я-Ё\s]', '', resp_) # удаление всего, что не является токенами из кириллических букв, в т.ч. знаков пунктуации

  return resp_

Тексты отзывов с рейтингами "5" и "1" поместим в словарь вида {текст отзыва: оценка отзыва}.

In [ ]:
clean_responses = {}

for el in tqdm(responses):
  if el['rating_grade'] == 5.0 or el['rating_grade'] == 1.0:
    clean_resp = cleaning(el['text'])
    clean_responses[clean_resp] = el['rating_grade']

100%|██████████| 153499/153499 [00:08<00:00, 18618.44it/s]


Из полученного словаря сформируем датафрейм.

In [ ]:
df = pd.DataFrame(list(clean_responses.items()), columns=['response', 'rating_grade'])
df.head()

,response,rating_grade
0,открыт вклад и счет в плюс к этому есть зарпл...,1
1,доброго времени вчера мне поступило смс уведом...,1
2,г около часов пришел в указанный офис с намер...,1
3,для оплаты коммунальных платежей пользуюсь пла...,1
4,в апреле этого года пришла в отделение сбербан...,1


In [ ]:
df['rating_grade'].unique()

array([1, 5])

На тестовое множество отведем 20% записей датафрейма df.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['response'], df['rating_grade'], test_size=0.2, random_state=42)

В исходном датасете классы отзывов с оценками "5" и "1" несбалансированны, негативных отзывов примерно в 3 раза больше, чем положительных:

In [ ]:
np.sum(y_train==1), np.sum(y_train==5), f'{37864/11813:.1f}'

(37864, 11813, '3.2')

In [ ]:
np.sum(y_test==1), np.sum(y_test==5), f'{9521/2899:.1f}'

(9521, 2899, '3.3')

**2. Используйте любой известный вам алгоритм классификации текстов для решения задачи и получите baseline. Сравните разные варианты векторизации текста: использование только униграм, пар или троек слов или с использованием символьных n-грам.**

В качестве классификатора будем использовать логистическую регрессию, в качестве векторизатора - CountVectorizer.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, f1_score

Сравнение разных вариантов векторизации текста.

Чтобы уменьшить влияние дисбаланса классов на качество модели, зададим параметр 'class_weight'='balanced' модели логистической регрессии.

In [ ]:
# Униграммы:
count_vect_1 = CountVectorizer(ngram_range=(1, 1))
bow_1 = count_vect_1.fit_transform(X_train)

In [ ]:
%%time
model_1 = LogisticRegression(max_iter=500, random_state=42, class_weight='balanced')
model_1.fit(bow_1, y_train)

CPU times: user 23.5 s, sys: 23.9 s, total: 47.4 s
Wall time: 27.2 s


LogisticRegression(class_weight='balanced', max_iter=500, random_state=42)

In [ ]:
pred_1 = model_1.predict(count_vect_1.transform(X_test))

In [ ]:
accuracy_1 = accuracy_score(y_test, pred_1).round(4)
f1_1 = f1_score(y_test, pred_1).round(4)
accuracy_1, f1_1

(0.9676, 0.9789)

In [ ]:
# Триграммы:
count_vect_3 = CountVectorizer(ngram_range=(3, 3))
bow_3 = count_vect_3.fit_transform(X_train)

In [ ]:
%%time
model_3 = LogisticRegression(max_iter=500, random_state=42, class_weight='balanced')
model_3.fit(bow_3, y_train)

CPU times: user 1min 17s, sys: 35.1 s, total: 1min 53s
Wall time: 1min 23s


LogisticRegression(class_weight='balanced', max_iter=500, random_state=42)

In [ ]:
pred_3 = model_3.predict(count_vect_3.transform(X_test))

In [ ]:
accuracy_3 = accuracy_score(y_test, pred_3).round(4)
f1_3 = f1_score(y_test, pred_3).round(4)
accuracy_3, f1_3

(0.9105, 0.944)

In [ ]:
# Символьные биграммы:
count_vect_ch = CountVectorizer(ngram_range=(2, 2), analyzer='char')
bow_ch = count_vect_ch.fit_transform(X_train)

In [ ]:
%%time
model_ch = LogisticRegression(max_iter=1000, solver='liblinear', random_state=42, class_weight='balanced')
model_ch.fit(bow_ch, y_train)

CPU times: user 1min 3s, sys: 475 ms, total: 1min 3s
Wall time: 1min 4s


LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42,
                   solver='liblinear')

In [ ]:
pred_ch = model_ch.predict(count_vect_ch.transform(X_test))

In [ ]:
accuracy_ch = accuracy_score(y_test, pred_ch).round(4)
f1_ch = f1_score(y_test, pred_ch).round(4)
accuracy_ch, f1_ch

(0.9198, 0.9464)

In [ ]:
df_bl = pd.DataFrame({'Vectorization':'CountVectorizer', 'n-grams':['1', '3', 'char'], 'accuracy':[accuracy_1, accuracy_3, accuracy_ch], 'f1':[f1_1, f1_3, f1_ch]})
df_bl

,Vectorization,n-grams,accuracy,f1
0,CountVectorizer,1,0.9676,0.9789
1,CountVectorizer,3,0.9105,0.9440
2,CountVectorizer,char,0.9198,0.9464


Лучшие значения метрик качества получены при использовании униграмм.

**3. Сравните, как изменяется качество решения задачи при использовании скрытых тем в качестве признаков:**
  *    **1-ый вариант: tf-idf-преобразование (sklearn.feature_extraction.text.TfidfTransformer) и сингулярное разложение (оно же – латентый семантический анализ) (sklearn.decomposition.TruncatedSVD),**
  *   **2-ой вариант: тематические модели LDA (sklearn.decomposition.LatentDirichletAllocation). Используйте accuracy и F-measure для оценки качества классификации.**

**В ноутбуке, размещенном в папке репозитория, написан примерный Pipeline для классификации текстов.**

**Эта часть задания может быть сделана с использованием sklearn.**

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD, LatentDirichletAllocation

1) Tfidf & TrncatedSVD.

In [ ]:
# Сократим количество признаков до 10:
tfidf_svd = make_pipeline(TfidfVectorizer(), TruncatedSVD(n_components=10, random_state=42))

In [ ]:
X_tfidf_svd = tfidf_svd.fit_transform(X_train)

In [ ]:
# Размерность массива признаков после применения TruncatedSVD:
X_tfidf_svd.shape

(49677, 10)

In [ ]:
%%time
model_lsa = LogisticRegression(max_iter=500, random_state=42, class_weight='balanced')
model_lsa.fit(X_tfidf_svd, y_train)

CPU times: user 82.6 ms, sys: 36 ms, total: 119 ms
Wall time: 65.3 ms


LogisticRegression(class_weight='balanced', max_iter=500, random_state=42)

In [ ]:
pred_lsa = model_lsa.predict(tfidf_svd.fit_transform(X_test))

In [ ]:
accuracy_lsa = accuracy_score(y_test, pred_lsa).round(4)
f1_lsa = f1_score(y_test, pred_lsa).round(4)
accuracy_lsa, f1_lsa

(0.8176, 0.8759)

2) CountVectorizer & LDA.

In [ ]:
cv_lda = make_pipeline(CountVectorizer(), LatentDirichletAllocation(n_components=10, random_state=42))

In [ ]:
%%time
X_cv_lda = cv_lda.fit_transform(X_train)

CPU times: user 6min 28s, sys: 36.6 s, total: 7min 5s
Wall time: 6min 33s


In [ ]:
X_cv_lda.shape

(49677, 10)

In [ ]:
%%time
model_lda = LogisticRegression(max_iter=500, random_state=42, class_weight='balanced')
model_lda.fit(X_cv_lda, y_train)

CPU times: user 127 ms, sys: 57.9 ms, total: 185 ms
Wall time: 102 ms


LogisticRegression(class_weight='balanced', max_iter=500, random_state=42)

In [ ]:
pred_lda = model_lda.predict(cv_lda.fit_transform(X_test))

In [ ]:
accuracy_lda = accuracy_score(y_test, pred_lda).round(4)
f1_lda = f1_score(y_test, pred_lda).round(4)
accuracy_lda, f1_lda

(0.6239, 0.7662)

Датафрейм со значениями метрик, полученных в данной части задания и лучшими значениями метрик предыдущей части:

In [ ]:
df_res = pd.DataFrame({'Vectorization':['CountVectorizer', 'Tfidf & TrncatedSVD', 'CountVectorizer & LDA'],
                       'accuracy':[accuracy_1, accuracy_lsa, accuracy_lda], 'f1':[f1_1, f1_lsa, f1_lda]})
df_res

,Vectorization,accuracy,f1
0,CountVectorizer,0.9676,0.9789
1,Tfidf & TrncatedSVD,0.8176,0.8759
2,CountVectorizer & LDA,0.6239,0.7662


При использовании скрытых тем в качестве признаков, метрики качества значительно снизились. Возможно, ситуацию может улучшить настройка параметров алгоритмов TruncatedSVD и LDA...